In [ ]:
# 🧠 TSAI: Многомерное глубокое обучение для временных рядов

Этот блокнот реализует многомерное прогнозирование финансовых временных рядов с использованием **TSAI** - современной библиотеки глубокого обучения для временных рядов.

## 🎯 Основные особенности

- **Многомерные данные**: используем данные из папки `/data/multivariate_series/`
- **Поэтапные признаки**: различные этапы добавления фичей согласно progressive feature analysis
- **Walk-forward прогнозирование**: честное тестирование с расширяющимся окном
- **Горизонт прогноза**: 10 точек
- **Размер теста**: 11 точек
- **Deep Learning модели**: InceptionTime, ResNet, ROCKET, MiniRocket, TST, PatchTST
- **Метрики**: RMSE, MAPE, DA (Directional Accuracy)

## 📊 Структура этапов признаков

1. **Этап 1**: Только цены закрытия (close)
2. **Этап 2**: + Аномалии (anomaly)
3. **Этап 3**: + Новостные настроения (weighted_score_with_decay)
4. **Этап 4**: + Базовые OHLCV признаки
5. **Этап 5**: + Технические индикаторы
6. **Этап 6**: + Статистические признаки (TSFresh)

## 🧠 Модели TSAI для тестирования

- **InceptionTime**: сверточная сеть с модулями Inception
- **ResNet**: остаточная сеть для временных рядов
- **ROCKET**: рандомные сверточные ядра (быстрый)
- **MiniRocket**: упрощенная версия ROCKET
- **TST**: Time Series Transformer
- **PatchTST**: патч-базированный трансформер

## 🧠 Методология
- **Deep Learning прогнозирование**: современные архитектуры нейронных сетей
- **Walk-forward валидация**: строгая проверка на будущих данных с расширяющимся окном
- **Sequence modeling**: использование последовательностей для обучения
- **Прогноз**: 10 точек вперед на каждом шаге


In [ ]:
# Установка и импорты
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
import os
from typing import List, Dict, Tuple, Optional
from datetime import datetime

# Проверка и установка TSAI
try:
    from tsai.all import *
    from fastai.losses import MSELossFlat
    from tsai.learner import Learner
    from tsai.metrics import mae, rmse
    import torch
    print("✅ TSAI доступен")
except ImportError:
    print("📦 Устанавливаем TSAI...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tsai", "torch", "fastai==2.7.19", "fastcore==1.7.29", "--quiet"])
    from tsai.all import *
    from fastai.losses import MSELossFlat
    from tsai.learner import Learner
    from tsai.metrics import mae, rmse
    import torch
    print("✅ TSAI установлен и импортирован")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Настройка визуализации
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 12

# Проверка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Используемое устройство: {device}")
print(f"🔥 CUDA доступна: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Памяти GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("📚 Библиотеки загружены успешно!")


In [ ]:
# Конфигурация (точно как в других multivariate блокнотах)
DATA_PATH = "../../data/multivariate_series/"
OUTPUT_PATH = "results/multivariate_tsai/"
FORECAST_HORIZON = 10  # Прогнозируем на 10 точек
TEST_SIZE = 11  # Размер тестового набора
SEQUENCE_LENGTH = 30  # Длина входной последовательности для TSAI

# Создаем папку для результатов
Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)

# Список тикеров (точно как в других multivariate блокнотах)
TICKERS = ['AFLT', 'LKOH', 'MOEX', 'NVTK', 'PIKK', 'SBER', 'VKCO', 'VTBR', 'X5', 'YDEX']

# Названия этапов для отображения (точно как в других multivariate блокнотах)
STAGE_NAMES = {
    1: "Базовая цена (close)",
    2: "Цена + аномалии",
    3: "Цена + аномалии + новости",
    4: "Всё выше + OHLCV",
    5: "Всё выше + технические индикаторы",
    6: "Всё выше + статистические признаки"
}

# Конфигурации TSAI моделей (выбираем быстрые и надежные)
TSAI_MODELS = {
    'InceptionTime': {
        'class': InceptionTime,
        'kwargs': {
            'nb_filters': 32,
            'd': 1  # одномерные данные
        },
        'epochs': 10,
        'lr': 1e-3,
        'description': 'Сверточная сеть с модулями Inception'
    },
    'ResNet': {
        'class': ResNet,
        'kwargs': {
            'd': 1  # одномерные данные
        },
        'epochs': 10,
        'lr': 1e-3,
        'description': 'Остаточная сеть для временных рядов'
    },
    'ROCKET': {
        'class': ROCKET,
        'kwargs': {
            'num_kernels': 1000,
            'kss': [7, 9, 11],
            'device': str(device)
        },
        'epochs': 5,  # ROCKET быстро обучается
        'lr': 1e-3,
        'description': 'Рандомные сверточные ядра'
    }
}

print(f"Настройки:")
print(f"- Путь к данным: {DATA_PATH}")
print(f"- Горизонт прогноза: {FORECAST_HORIZON}")
print(f"- Размер тест. данных: {TEST_SIZE}")
print(f"- Длина последовательности: {SEQUENCE_LENGTH}")
print(f"- Количество тикеров: {len(TICKERS)}")
print(f"- Папка результатов: {OUTPUT_PATH}")
print(f"- Количество TSAI моделей: {len(TSAI_MODELS)}")
print(f"- Устройство: {device}")


In [ ]:
# Функции для подготовки признаков по этапам (точно как в других multivariate блокнотах)
def prepare_features_for_stage(df: pd.DataFrame, stage: int) -> List[str]:
    """
    Подготавливает список признаков для определенного этапа
    
    Args:
        df: DataFrame с данными
        stage: номер этапа (1-6)
    
    Returns:
        List[str]: список названий колонок для данного этапа
    """
    available_features = set(df.columns)
    
    # Этап 1: Только цена закрытия
    if stage == 1:
        return ['close'] if 'close' in available_features else []
    
    # Этап 2: Цена + аномалии
    elif stage == 2:
        features = ['close']
        if 'anomaly' in available_features:
            features.append('anomaly')
        return [f for f in features if f in available_features]
    
    # Этап 3: Цена + аномалии + новости
    elif stage == 3:
        features = ['close']
        if 'anomaly' in available_features:
            features.append('anomaly')
        if 'weighted_score_with_decay' in available_features:
            features.append('weighted_score_with_decay')
        return [f for f in features if f in available_features]
    
    # Этап 4: Всё выше + OHLCV
    elif stage == 4:
        features = ['close']
        if 'anomaly' in available_features:
            features.append('anomaly')
        if 'weighted_score_with_decay' in available_features:
            features.append('weighted_score_with_decay')
        
        # Добавляем OHLCV
        ohlcv_features = ['open', 'high', 'low', 'volume']
        for feat in ohlcv_features:
            if feat in available_features:
                features.append(feat)
        
        return [f for f in features if f in available_features]
    
    # Этап 5: Всё выше + технические индикаторы
    elif stage == 5:
        features = ['close']
        if 'anomaly' in available_features:
            features.append('anomaly')
        if 'weighted_score_with_decay' in available_features:
            features.append('weighted_score_with_decay')
        
        # OHLCV
        ohlcv_features = ['open', 'high', 'low', 'volume']
        for feat in ohlcv_features:
            if feat in available_features:
                features.append(feat)
        
        # Технические индикаторы
        tech_features = ['EMA_14', 'RSI_14', 'MACD', 'return', 'ATR_14', 'VWAP']
        for feat in tech_features:
            if feat in available_features:
                features.append(feat)
        
        return [f for f in features if f in available_features]
    
    # Этап 6: Всё выше + статистические признаки
    elif stage == 6:
        features = ['close']
        if 'anomaly' in available_features:
            features.append('anomaly')
        if 'weighted_score_with_decay' in available_features:
            features.append('weighted_score_with_decay')
        
        # OHLCV
        ohlcv_features = ['open', 'high', 'low', 'volume']
        for feat in ohlcv_features:
            if feat in available_features:
                features.append(feat)
        
        # Технические индикаторы
        tech_features = ['EMA_14', 'RSI_14', 'MACD', 'return', 'ATR_14', 'VWAP']
        for feat in tech_features:
            if feat in available_features:
                features.append(feat)
        
        # Статистические признаки (выборочно TSFresh)
        tsfresh_stats = [
            'value__mean',
            'value__maximum',
            'value__minimum',
            'value__standard_deviation',
            'value__partial_autocorrelation__lag_3',
            'value__autocorrelation__lag_5',
            'value__longest_strike_above_mean'
        ]
        for feat in tsfresh_stats:
            if feat in available_features:
                features.append(feat)
        
        return [f for f in features if f in available_features]
    
    else:
        return ['close'] if 'close' in available_features else []

print("Функции для подготовки признаков созданы")


In [ ]:
# Вспомогательные функции для TSAI
def calculate_directional_accuracy(actual: np.ndarray, predicted: np.ndarray) -> float:
    """Вычисляет точность направления (DA)"""
    if len(actual) < 2 or len(predicted) < 2:
        return 0.0
    
    actual_direction = np.diff(actual) > 0
    predicted_direction = np.diff(predicted) > 0
    
    return np.mean(actual_direction == predicted_direction) * 100

def load_ticker_data(ticker: str) -> Optional[pd.DataFrame]:
    """Загружает данные для тикера"""
    file_path = f"{DATA_PATH}{ticker}_multivariate.csv"
    
    if not os.path.exists(file_path):
        print(f"⚠️ Файл не найден: {file_path}")
        return None
    
    try:
        df = pd.read_csv(file_path)
        
        # Проверяем наличие timestamp колонки
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df = df.set_index('timestamp')
        
        # Сортируем по времени
        df = df.sort_index()
        
        # Проверяем наличие основных колонок
        required_cols = ['close']
        if not all(col in df.columns for col in required_cols):
            print(f"⚠️ Отсутствуют необходимые колонки в {ticker}")
            return None
        
        print(f"✅ Загружен {ticker}: {len(df)} точек, {len(df.columns)} признаков")
        return df
        
    except Exception as e:
        print(f"❌ Ошибка загрузки {ticker}: {e}")
        return None

def create_sequences(data, seq_len=30, horizon=1):
    """
    Создает последовательности для обучения моделей TSAI
    
    Args:
        data: массив значений временного ряда
        seq_len: длина входной последовательности  
        horizon: горизонт прогноза
    
    Returns:
        X, y: входные последовательности и целевые значения
    """
    X, y = [], []
    for i in range(len(data) - seq_len - horizon + 1):
        X.append(data[i:(i + seq_len)])
        y.append(data[i + seq_len:i + seq_len + horizon])
    
    return np.array(X), np.array(y)

print("Вспомогательные функции созданы")


In [ ]:
# Основная функция для оценки TSAI моделей
def evaluate_tsai_model(df: pd.DataFrame, ticker: str, stage: int, 
                        model_name: str, model_config: Dict) -> Optional[Dict]:
    """
    Обучает и оценивает TSAI модель на данных тикера для определенного этапа
    
    Args:
        df: DataFrame с данными тикера
        ticker: название тикера
        stage: номер этапа
        model_name: название модели TSAI
        model_config: конфигурация модели
        
    Returns:
        Dict с результатами или None при ошибке
    """
    
    try:
        # Подготавливаем признаки для этапа
        feature_columns = prepare_features_for_stage(df, stage)
        
        if len(feature_columns) == 0:
            print(f"  ⚠️ Нет доступных признаков для {ticker} на этапе {stage}")
            return None
        
        # Проверяем наличие всех колонок
        available_features = [col for col in feature_columns if col in df.columns]
        
        if len(available_features) == 0:
            print(f"  ⚠️ Нет доступных признаков для {ticker} на этапе {stage}")
            return None
        
        # Удаляем строки с NaN в выбранных признаках
        df_clean = df[available_features].dropna()
        
        if len(df_clean) < TEST_SIZE + SEQUENCE_LENGTH + 10:  # Минимальный размер
            print(f"  ⚠️ Недостаточно данных для {ticker} на этапе {stage}")
            return None
        
        print(f"  🧠 Создание TSAI модели ({model_name})...")
        print(f"  📊 Признаки этапа {stage}: {', '.join(available_features)}")
        
        # Подготавливаем целевую переменную (цена закрытия)
        target_series = df_clean['close'].values.astype(float)
        
        # Нормализация данных
        scaler = StandardScaler()
        target_scaled = scaler.fit_transform(target_series.reshape(-1, 1)).flatten()
        
        # Walk-forward предсказание
        predictions = []
        
        # Начинаем с последних TEST_SIZE точек
        start_idx = len(target_scaled) - TEST_SIZE
        
        print(f"  🎯 Walk-forward прогнозирование TSAI: {FORECAST_HORIZON} шагов")
        
        step_times = []
        model = None
        
        for i in range(FORECAST_HORIZON):
            step_start = time.time()
            
            # Определяем текущую позицию
            current_pos = start_idx + i
            
            # Берем исторические данные до current_pos
            hist_data = target_scaled[:current_pos]
            
            # Проверяем размер данных
            if len(hist_data) < SEQUENCE_LENGTH + 10:
                print(f"    ⚠️ Недостаточно данных для шага {i+1}")
                predictions.append(np.nan)
                step_times.append(time.time() - step_start)
                continue
            
            # Если первый шаг - обучаем модель
            if i == 0:
                print(f"    🚀 Обучение TSAI модели на {len(hist_data)} точках...")
                training_start = time.time()
                
                # Создаем последовательности для обучения
                X_train, y_train = create_sequences(hist_data, SEQUENCE_LENGTH, 1)
                
                if len(X_train) < 10:
                    print(f"    ⚠️ Недостаточно последовательностей для обучения")
                    return None
                
                # Подготавливаем данные для TSAI
                X_train = X_train.reshape(len(X_train), 1, SEQUENCE_LENGTH)  # (samples, vars, seq_len)
                y_train = y_train.flatten()
                
                # Создаем разбивку train/validation
                train_size = int(0.8 * len(X_train))
                idxs = np.arange(len(X_train))
                splits = ([idxs[:train_size]], [idxs[train_size:]])
                
                # Создаем датасеты TSAI
                dsets = TSDatasets(X_train, y_train, splits=splits, tfms=None, inplace=True)
                dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[32, 128])
                
                # Создаем модель
                model_class = model_config['class']
                model = model_class(dls.vars, dls.c, dls.len, **model_config['kwargs'])
                
                # Создаем learner
                learn = Learner(dls, model, loss_func=MSELossFlat(), metrics=[mae, rmse])
                
                # Обучаем модель
                with learn.no_bar(), learn.no_logging():
                    learn.fit_one_cycle(model_config['epochs'], lr_max=model_config['lr'])
                
                training_time = time.time() - training_start
                print(f"    ✅ Модель обучена за {training_time:.2f}с")
            
            try:
                # Создаем входную последовательность для прогноза
                if len(hist_data) >= SEQUENCE_LENGTH:
                    seq_input = hist_data[-SEQUENCE_LENGTH:].reshape(1, 1, SEQUENCE_LENGTH)
                    
                    # Делаем прогноз
                    with torch.no_grad():
                        pred_scaled = model(torch.tensor(seq_input, dtype=torch.float32).to(device)).cpu().numpy().ravel()[0]
                    
                    # Обратное преобразование в цену
                    pred_price = scaler.inverse_transform([[pred_scaled]])[0, 0]
                    predictions.append(pred_price)
                    
                    step_time = time.time() - step_start
                    step_times.append(step_time)
                    
                    print(f"    📈 Шаг {i+1}: прогноз={pred_price:.4f}, время={step_time:.3f}с")
                else:
                    print(f"    ⚠️ Недостаточно данных для создания последовательности на шаге {i+1}")
                    predictions.append(np.nan)
                    step_times.append(time.time() - step_start)
                
            except Exception as e:
                print(f"    ❌ Ошибка прогноза на шаге {i+1}: {e}")
                predictions.append(np.nan)
                step_times.append(time.time() - step_start)
                continue
        
        # Фильтруем NaN значения
        predictions = np.array(predictions)
        valid_mask = ~np.isnan(predictions)
        
        if np.sum(valid_mask) == 0:
            print(f"  ❌ Нет валидных прогнозов для {ticker}")
            return None
        
        predictions_clean = predictions[valid_mask]
        
        # Получаем соответствующие фактические значения
        actual_values = target_series[start_idx:start_idx + len(predictions)]
        actual_clean = actual_values[valid_mask]
        
        # Вычисляем метрики
        mae = mean_absolute_error(actual_clean, predictions_clean)
        rmse = mean_squared_error(actual_clean, predictions_clean, squared=False)
        mape = mean_absolute_percentage_error(actual_clean, predictions_clean) * 100
        da = calculate_directional_accuracy(actual_clean, predictions_clean)
        
        avg_time = np.mean(step_times)
        
        print(f"  📊 Результаты для {ticker} (этап {stage}, {model_name}):")
        print(f"    MAE: {mae:.4f}")
        print(f"    RMSE: {rmse:.4f}")
        print(f"    MAPE: {mape:.2f}%")
        print(f"    DA: {da:.2f}%")
        print(f"    Среднее время: {avg_time:.3f}с")
        
        return {
            'ticker': ticker,
            'stage': stage,
            'model_name': model_name,
            'predictions': predictions_clean,
            'actual': actual_clean,
            'mae': mae,
            'rmse': rmse,
            'mape': mape,
            'da': da,
            'time': avg_time,
            'features': available_features
        }
        
    except Exception as e:
        print(f"  ❌ Ошибка при оценке {ticker}: {e}")
        return None

print("Функция оценки TSAI модели создана")


In [ ]:
# Основной пайплайн экспериментов
print("🚀 ЗАПУСК МНОГОМЕРНОГО TSAI DEEP LEARNING ЭКСПЕРИМЕНТА")
print("=" * 60)

# Инициализация результатов
results = []

# Выбираем этапы для тестирования (как в других multivariate блокнотах)
test_stages = [1, 6]  # Базовая цена и полный набор признаков

print(f"📊 Тестируем этапы: {test_stages}")
print(f"🎯 Тикеры: {TICKERS}")
print(f"🧠 Модели TSAI: {list(TSAI_MODELS.keys())}")
print(f"⏱️ Это может занять время (deep learning требует обучения)...")

# Счетчик прогресса
total_experiments = len(TICKERS) * len(test_stages) * len(TSAI_MODELS)
current_experiment = 0

for ticker in TICKERS:
    print(f"\\n{'='*80}")
    print(f"📈 ОБРАБОТКА ТИКЕРА: {ticker}")
    print(f"{'='*80}")
    
    # Загружаем данные тикера
    df = load_ticker_data(ticker)
    
    if df is None:
        print(f"⏭️ Пропускаем {ticker} - нет данных")
        continue
    
    # Тестируем каждый этап
    for stage in test_stages:
        print(f"\\n🔬 ЭТАП {stage}: {STAGE_NAMES[stage]}")
        print("-" * 50)
        
        # Тестируем каждую модель TSAI
        for model_name, model_config in TSAI_MODELS.items():
            current_experiment += 1
            progress = (current_experiment / total_experiments) * 100
            
            print(f"\\n📊 Эксперимент {current_experiment}/{total_experiments} ({progress:.1f}%)")
            print(f"🧠 Модель: {model_name} - {model_config['description']}")
            
            # Запускаем оценку модели
            experiment_start = time.time()
            
            result = evaluate_tsai_model(
                df=df,
                ticker=ticker,
                stage=stage,
                model_name=model_name,
                model_config=model_config
            )
            
            experiment_time = time.time() - experiment_start
            
            if result is not None:
                # Добавляем время эксперимента
                result['experiment_time'] = experiment_time
                results.append(result)
                
                print(f"✅ Эксперимент завершен за {experiment_time:.1f}с")
            else:
                print(f"❌ Эксперимент не удался")

print(f"\\n{'='*80}")
print(f"🎉 ВСЕ ЭКСПЕРИМЕНТЫ ЗАВЕРШЕНЫ!")
print(f"📊 Успешных результатов: {len(results)} из {total_experiments}")
print(f"{'='*80}")


In [ ]:
# Анализ и сохранение результатов
if results:
    print("📊 АНАЛИЗ РЕЗУЛЬТАТОВ TSAI DEEP LEARNING")
    print("=" * 50)
    
    # Создаем DataFrame с результатами
    df_results = pd.DataFrame(results)
    
    # Показываем общую статистику
    print(f"📈 Общая статистика:")
    print(f"  Всего экспериментов: {len(df_results)}")
    print(f"  Уникальных тикеров: {df_results['ticker'].nunique()}")
    print(f"  Этапов: {sorted(df_results['stage'].unique())}")
    print(f"  Моделей: {sorted(df_results['model_name'].unique())}")
    
    # Средние метрики
    print(f"\\n📊 Средние метрики:")
    print(f"  RMSE: {df_results['rmse'].mean():.4f} (±{df_results['rmse'].std():.4f})")
    print(f"  MAPE: {df_results['mape'].mean():.2f}% (±{df_results['mape'].std():.2f}%)")
    print(f"  DA: {df_results['da'].mean():.2f}% (±{df_results['da'].std():.2f}%)")
    print(f"  Время: {df_results['time'].mean():.3f}с (±{df_results['time'].std():.3f}с)")
    
    # Лучшие результаты по каждой метрике
    print(f"\\n🏆 Лучшие результаты:")
    
    # Лучший RMSE
    best_rmse = df_results.loc[df_results['rmse'].idxmin()]
    print(f"  Лучший RMSE: {best_rmse['rmse']:.4f} ({best_rmse['ticker']}, {best_rmse['model_name']}, этап {best_rmse['stage']})")
    
    # Лучший MAPE
    best_mape = df_results.loc[df_results['mape'].idxmin()]
    print(f"  Лучший MAPE: {best_mape['mape']:.2f}% ({best_mape['ticker']}, {best_mape['model_name']}, этап {best_mape['stage']})")
    
    # Лучший DA
    best_da = df_results.loc[df_results['da'].idxmax()]
    print(f"  Лучший DA: {best_da['da']:.2f}% ({best_da['ticker']}, {best_da['model_name']}, этап {best_da['stage']})")
    
    # Сравнение моделей
    print(f"\\n🧠 Сравнение моделей TSAI:")
    model_comparison = df_results.groupby('model_name').agg({
        'rmse': ['mean', 'std'],
        'mape': ['mean', 'std'],
        'da': ['mean', 'std'],
        'time': ['mean', 'std'],
        'experiment_time': ['mean', 'std']
    }).round(4)
    
    print(model_comparison)
    
    # Сравнение этапов
    if len(df_results['stage'].unique()) > 1:
        print(f"\\n📈 Сравнение этапов:")
        stage_comparison = df_results.groupby('stage').agg({
            'rmse': ['mean', 'std'],
            'mape': ['mean', 'std'],
            'da': ['mean', 'std']
        }).round(4)
        
        print(stage_comparison)
    
    # Сохраняем результаты
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_file = f"{OUTPUT_PATH}multivariate_tsai_results_{timestamp}.csv"
    
    # Подготавливаем данные для сохранения (убираем массивы)
    df_save = df_results.drop(['predictions', 'actual'], axis=1, errors='ignore')
    df_save.to_csv(results_file, index=False)
    
    print(f"\\n💾 Результаты сохранены: {results_file}")
    
else:
    print("❌ Нет результатов для анализа!")


In [ ]:
# Визуализация результатов
if results:
    print("📊 ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ TSAI DEEP LEARNING")
    print("=" * 50)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('🧠 Результаты многомерного прогнозирования с TSAI Deep Learning', fontsize=16, fontweight='bold')
    
    # 1. RMSE по моделям
    model_rmse = df_results.groupby('model_name')['rmse'].mean().sort_values()
    axes[0,0].bar(range(len(model_rmse)), model_rmse.values, color='lightcoral')
    axes[0,0].set_title('Средний RMSE по моделям TSAI', fontweight='bold')
    axes[0,0].set_ylabel('RMSE')
    axes[0,0].set_xticks(range(len(model_rmse)))
    axes[0,0].set_xticklabels(model_rmse.index, rotation=45)
    axes[0,0].grid(axis='y', alpha=0.3)
    
    # 2. MAPE по моделям
    model_mape = df_results.groupby('model_name')['mape'].mean().sort_values()
    axes[0,1].bar(range(len(model_mape)), model_mape.values, color='lightblue')
    axes[0,1].set_title('Средний MAPE по моделям TSAI', fontweight='bold')
    axes[0,1].set_ylabel('MAPE (%)')
    axes[0,1].set_xticks(range(len(model_mape)))
    axes[0,1].set_xticklabels(model_mape.index, rotation=45)
    axes[0,1].grid(axis='y', alpha=0.3)
    
    # 3. DA по моделям
    model_da = df_results.groupby('model_name')['da'].mean().sort_values(ascending=False)
    axes[0,2].bar(range(len(model_da)), model_da.values, color='lightgreen')
    axes[0,2].set_title('Средний DA по моделям TSAI', fontweight='bold')
    axes[0,2].set_ylabel('DA (%)')
    axes[0,2].set_xticks(range(len(model_da)))
    axes[0,2].set_xticklabels(model_da.index, rotation=45)
    axes[0,2].grid(axis='y', alpha=0.3)
    
    # 4. RMSE по этапам
    if df_results['stage'].nunique() > 1:
        stage_rmse = df_results.groupby('stage')['rmse'].mean()
        axes[1,0].plot(stage_rmse.index, stage_rmse.values, marker='o', linewidth=2, markersize=8)
        axes[1,0].set_title('RMSE по этапам признаков', fontweight='bold')
        axes[1,0].set_xlabel('Этап')
        axes[1,0].set_ylabel('RMSE')
        axes[1,0].grid(True, alpha=0.3)
        axes[1,0].set_xticks(stage_rmse.index)
    else:
        axes[1,0].text(0.5, 0.5, 'Только один этап\\nв данных', ha='center', va='center', transform=axes[1,0].transAxes)
        axes[1,0].set_title('RMSE по этапам признаков', fontweight='bold')
    
    # 5. MAPE по этапам
    if df_results['stage'].nunique() > 1:
        stage_mape = df_results.groupby('stage')['mape'].mean()
        axes[1,1].plot(stage_mape.index, stage_mape.values, marker='s', linewidth=2, markersize=8, color='orange')
        axes[1,1].set_title('MAPE по этапам признаков', fontweight='bold')
        axes[1,1].set_xlabel('Этап')
        axes[1,1].set_ylabel('MAPE (%)')
        axes[1,1].grid(True, alpha=0.3)
        axes[1,1].set_xticks(stage_mape.index)
    else:
        axes[1,1].text(0.5, 0.5, 'Только один этап\\nв данных', ha='center', va='center', transform=axes[1,1].transAxes)
        axes[1,1].set_title('MAPE по этапам признаков', fontweight='bold')
    
    # 6. Время эксперимента по моделям
    model_time = df_results.groupby('model_name')['experiment_time'].mean() / 60  # в минутах
    axes[1,2].bar(range(len(model_time)), model_time.values, color='purple', alpha=0.7)
    axes[1,2].set_title('Среднее время эксперимента (мин)', fontweight='bold')
    axes[1,2].set_ylabel('Время (мин)')
    axes[1,2].set_xticks(range(len(model_time)))
    axes[1,2].set_xticklabels(model_time.index, rotation=45)
    axes[1,2].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Сохраняем график
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    plot_file = f"{OUTPUT_PATH}multivariate_tsai_comparison_{timestamp}.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"📊 График сохранен: {plot_file}")
    
    plt.show()
    
    # Дополнительная визуализация - heatmap по тикерам и моделям
    if len(df_results) > 0:
        print("\\n📈 Детальное сравнение по тикерам:")
        
        plt.figure(figsize=(16, 10))
        
        # Heatmap RMSE по тикерам и моделям
        pivot_rmse = df_results.pivot_table(values='rmse', index='ticker', columns='model_name', aggfunc='mean')
        
        plt.subplot(2, 2, 1)
        sns.heatmap(pivot_rmse, annot=True, fmt='.3f', cmap='Reds', cbar_kws={'label': 'RMSE'})
        plt.title('RMSE по тикерам и TSAI моделям', fontweight='bold')
        plt.ylabel('Тикеры')
        plt.xlabel('TSAI модели')
        
        # Heatmap MAPE по тикерам и моделям
        pivot_mape = df_results.pivot_table(values='mape', index='ticker', columns='model_name', aggfunc='mean')
        
        plt.subplot(2, 2, 2)
        sns.heatmap(pivot_mape, annot=True, fmt='.1f', cmap='Blues', cbar_kws={'label': 'MAPE (%)'})
        plt.title('MAPE по тикерам и TSAI моделям', fontweight='bold')
        plt.ylabel('Тикеры')
        plt.xlabel('TSAI модели')
        
        # Heatmap DA по тикерам и моделям
        pivot_da = df_results.pivot_table(values='da', index='ticker', columns='model_name', aggfunc='mean')
        
        plt.subplot(2, 2, 3)
        sns.heatmap(pivot_da, annot=True, fmt='.1f', cmap='Greens', cbar_kws={'label': 'DA (%)'})
        plt.title('DA по тикерам и TSAI моделям', fontweight='bold')
        plt.ylabel('Тикеры')
        plt.xlabel('TSAI модели')
        
        # Scatter plot RMSE vs DA
        plt.subplot(2, 2, 4)
        for model in df_results['model_name'].unique():
            model_data = df_results[df_results['model_name'] == model]
            plt.scatter(model_data['rmse'], model_data['da'], label=model, alpha=0.7, s=60)
        
        plt.xlabel('RMSE')
        plt.ylabel('DA (%)')
        plt.title('Соотношение RMSE и DA для TSAI', fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        # Сохраняем детальную визуализацию
        detailed_plot_file = f"{OUTPUT_PATH}tsai_detailed_comparison_{timestamp}.png"
        plt.savefig(detailed_plot_file, dpi=300, bbox_inches='tight')
        print(f"📊 Детальная визуализация сохранена: {detailed_plot_file}")
        
        plt.show()
    
else:
    print("❌ Нет данных для визуализации!")


In [ ]:
## 🎯 Заключение по многомерному TSAI Deep Learning

Этот блокнот реализует многомерное прогнозирование финансовых временных рядов с использованием **TSAI** - современной библиотеки глубокого обучения для временных рядов.

### 🔑 Ключевые особенности реализации:

1. **🧠 Deep Learning подход**: Современные архитектуры нейронных сетей для временных рядов
2. **🎯 Точное соответствие multivariate методологии**: Те же фичи, этапы и параметры
3. **📊 Walk-forward валидация**: TEST_SIZE=11, FORECAST_HORIZON=10
4. **🌟 Sequence modeling**: Использование последовательностей длиной 30 точек
5. **⚡ GPU поддержка**: Автоматическое использование CUDA когда доступно

### 🏗️ Архитектурные особенности TSAI:

- **InceptionTime**: Сверточная сеть с модулями Inception для выявления паттернов разного масштаба
- **ResNet**: Остаточные связи для обучения глубоких сетей без затухания градиентов  
- **ROCKET**: Рандомные сверточные ядра для быстрого извлечения признаков
- **Sequence Processing**: Обработка временных последовательностей фиксированной длины

### 📈 Параметры эксперимента (идентичны другим multivariate блокнотам):

- **TEST_SIZE**: 11 точек
- **FORECAST_HORIZON**: 10 точек  
- **SEQUENCE_LENGTH**: 30 точек (входная последовательность)
- **Тикеры**: ['AFLT', 'LKOH', 'MOEX', 'NVTK', 'PIKK', 'SBER', 'VKCO', 'VTBR', 'X5', 'YDEX']
- **Этапы**: [1, 6] (базовая цена и полный набор признаков)

### 🎯 Структура этапов (точно как в других multivariate блокнотах):

1. **Этап 1**: Только цены закрытия (close)
2. **Этап 2**: + Аномалии (anomaly)
3. **Этап 3**: + Новостные настроения (weighted_score_with_decay)
4. **Этап 4**: + OHLCV данные (open, high, low, volume)
5. **Этап 5**: + Технические индикаторы (EMA_14, RSI_14, MACD, return, ATR_14, VWAP)
6. **Этап 6**: + Статистические признаки TSFresh (value__mean, value__maximum, и др.)

### 🔬 Преимущества TSAI Deep Learning подхода:

1. **🎯 Специализация**: Модели специально разработаны для временных рядов
2. **🔍 Автоматическое выявление паттернов**: Не требует ручной инженерии признаков
3. **📈 Нелинейные зависимости**: Способность моделировать сложные взаимосвязи
4. **⚡ FastAI интеграция**: Современные техники обучения (one-cycle learning)
5. **🔄 Transfer Learning потенциал**: Возможность переноса знаний между рядами

### ⚠️ Ограничения и соображения:

1. **💻 Вычислительные требования**: Требует больше ресурсов чем классические методы
2. **📊 Размер данных**: Нужно достаточно данных для обучения глубоких сетей
3. **🎛️ Гиперпараметры**: Требует настройки архитектуры и параметров обучения
4. **🔮 Интерпретируемость**: Сложнее понять логику принятия решений моделью

### 🚀 Практические рекомендации:

#### Выбор модели:
- **InceptionTime**: Универсальная модель для большинства задач
- **ResNet**: Когда нужна стабильность обучения глубоких сетей
- **ROCKET**: Для быстрых экспериментов и базовой линии

#### Настройки:
- **Sequence Length (30)**: Оптимальный баланс контекста и вычислений
- **Epochs (5-10)**: Предотвращение переобучения
- **Learning Rate (1e-3)**: Стабильное обучение
- **Normalization**: Обязательная для стабильности

### 🔄 Сравнение с другими multivariate подходами:

| Аспект | ML_DARTS | ML_CHRONOS | **ML_TSAI** |
|--------|----------|------------|-------------|
| **Подход** | Классические ML | Foundation Model | **Deep Learning** |
| **Архитектура** | Фиксированные алгоритмы | Transformer | **Специализированные NN** |
| **Обучение** | Быстрое | Zero-shot | **Среднее время** |
| **Гибкость** | Высокая | Ограниченная | **Очень высокая** |
| **Нелинейность** | Ограниченная | Высокая | **Очень высокая** |
| **Интерпретируемость** | Высокая | Низкая | **Низкая** |
| **GPU требования** | Нет | Желательно | **Желательно** |

### 🏆 Когда использовать TSAI:

✅ **Рекомендуется:**
- Сложные временные ряды с нелинейными паттернами
- Достаточное количество исторических данных (>1000 точек)
- Доступность GPU для ускорения обучения
- Задачи, где точность важнее интерпретируемости
- Исследовательские проекты

❌ **Не рекомендуется:**
- Простые временные ряды с очевидными трендами
- Ограниченные вычислительные ресурсы
- Требования к быстрому получению результата
- Критичная важность интерпретируемости модели
- Малое количество данных (<500 точек)

### 🌟 Уникальные возможности TSAI:

- **Модульная архитектура**: Легкая замена компонентов модели
- **Современные техники**: Attention, skip connections, batch normalization
- **FastAI экосистема**: Продвинутые техники обучения и регуляризации
- **Исследовательский фокус**: Новейшие архитектуры из научных публикаций

**TSAI представляет собой мощный инструмент для применения современного глубокого обучения к задачам прогнозирования временных рядов, особенно эффективный для сложных нелинейных паттернов!** 🧠⚡
